# This file simply takes into account all instructions found in every program of the experiments and setsup a tokenizer for it.

In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import random
import tensorflow as tf
import numpy as np
import timeit
from sklearn import metrics
import scipy.io as sio
import csv
import os
import math 
import random as rand
from numpy import genfromtxt
from numpy import sum,isrealobj,sqrt
from numpy.random import standard_normal

#import torch
#from torch import nn

import warnings
from scipy.spatial import distance


from importlib import reload
import sys
sys.path.append("../Imports/") # Adds higher directory to python modules path.

from Modules.Tools import MapTool
from Modules.Tools import BoxPlot
from Modules.Tools import Extractor
from Modules.Tools import peakCorrelation as pC

# Notebook Parameters
- this is simply for obtaining the graphs with manual mapping. Pad is closely relate to the sampling rate for the EM signal

In [22]:
load_folder = "../Datasets/Instruction_Code/"
save_folder = "../Datasets/Tokenizer/"
pad = 8
y_low=-2.8
y_high=2.8
amount=3000

# Tokenizer Setup

We need to be able to convert code from either string or a code document into a computer/model readable format.
This can create several challenges.
- The amount of code can vary between programs.
- The signal size can vary.
- The embedded instructions sould be unique like bagging words.
- Note that words for each instruction mustnt be spaced, hypened, unscored or contain any unique lettering or symbol as this will split the word/instruction into different tokens.

Thus we will limit the signal size and amount of instructions to a set amount to generate the program in sections for all signals.
Consequently, we will create a precise convertion from code to numbers, where each number will represent a unique instruction (i.e., tokenization).

Later work my switch from tokenizer to something else, however, tokenizer is a simple method that fullfills the requirments.

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=100, char_level=False)

In [12]:
Base_Benign_Code = np.load(load_folder + "Base_Benign.npy").tolist()
Base_Anom_Code = np.load(load_folder + "Base_Anom.npy").tolist()

In [7]:
Update_50_A_Benign_Code = np.load(load_folder + "Update_50_A_Benign.npy").tolist()
Update_50_B_Benign_Code = np.load(load_folder + "Update_50_B_Benign.npy").tolist()
Update_50_C_Benign_Code = np.load(load_folder + "Update_50_C_Benign.npy").tolist()

Update_50_A_Anom_Code = np.load(load_folder + "Update_50_A_Anom.npy").tolist()
Update_50_B_Anom_Code = np.load(load_folder + "Update_50_B_Anom.npy").tolist()
Update_50_C_Anom_Code = np.load(load_folder + "Update_50_C_Anom.npy").tolist()

In [8]:
Update_75_A_Benign_Code = np.load(load_folder + "Update_75_A_Benign.npy").tolist()
Update_75_B_Benign_Code = np.load(load_folder + "Update_75_B_Benign.npy").tolist()
Update_75_C_Benign_Code = np.load(load_folder + "Update_75_C_Benign.npy").tolist()

Update_75_A_Anom_Code = np.load(load_folder + "Update_75_A_Anom.npy").tolist()
Update_75_B_Anom_Code = np.load(load_folder + "Update_75_B_Anom.npy").tolist()
Update_75_C_Anom_Code = np.load(load_folder + "Update_75_C_Anom.npy").tolist()

In [9]:
Update_100_A_Benign_Code = np.load(load_folder + "Update_100_A_Benign.npy").tolist()
Update_100_B_Benign_Code = np.load(load_folder + "Update_100_B_Benign.npy").tolist()
Update_100_C_Benign_Code = np.load(load_folder + "Update_100_C_Benign.npy").tolist()

Update_100_A_Anom_Code = np.load(load_folder + "Update_100_A_Anom.npy").tolist()
Update_100_B_Anom_Code = np.load(load_folder + "Update_100_B_Anom.npy").tolist()
Update_100_C_Anom_Code = np.load(load_folder + "Update_100_C_Anom.npy").tolist()

In [10]:
Train_Program_1_Code = np.load(load_folder + "Train_Program_1.npy").tolist()
Train_Program_2_Code = np.load(load_folder + "Train_Program_2.npy").tolist()
Train_Program_3_Code = np.load(load_folder + "Train_Program_3.npy").tolist()
Train_Program_4_Code = np.load(load_folder + "Train_Program_4.npy").tolist()

In [13]:
all_code = np.copy(Base_Benign_Code)
all_code = np.concatenate((all_code, Base_Anom_Code))

all_code = np.concatenate((all_code, Update_50_A_Benign_Code))
all_code = np.concatenate((all_code, Update_50_B_Benign_Code))
all_code = np.concatenate((all_code, Update_50_C_Benign_Code))
all_code = np.concatenate((all_code, Update_50_A_Anom_Code))
all_code = np.concatenate((all_code, Update_50_B_Anom_Code))
all_code = np.concatenate((all_code, Update_50_C_Anom_Code))

all_code = np.concatenate((all_code, Update_75_A_Benign_Code))
all_code = np.concatenate((all_code, Update_75_B_Benign_Code))
all_code = np.concatenate((all_code, Update_75_C_Benign_Code))
all_code = np.concatenate((all_code, Update_75_A_Anom_Code))
all_code = np.concatenate((all_code, Update_75_B_Anom_Code))
all_code = np.concatenate((all_code, Update_75_C_Anom_Code))

all_code = np.concatenate((all_code, Update_100_A_Benign_Code))
all_code = np.concatenate((all_code, Update_100_B_Benign_Code))
all_code = np.concatenate((all_code, Update_100_C_Benign_Code))
all_code = np.concatenate((all_code, Update_100_A_Anom_Code))
all_code = np.concatenate((all_code, Update_100_B_Anom_Code))
all_code = np.concatenate((all_code, Update_100_C_Anom_Code))

all_code = np.concatenate((all_code, Train_Program_1_Code))
all_code = np.concatenate((all_code, Train_Program_2_Code))
all_code = np.concatenate((all_code, Train_Program_3_Code))
all_code = np.concatenate((all_code, Train_Program_4_Code))

In [14]:
all_code_unique = np.unique(all_code)
print(all_code_unique)

['adc' 'add' 'brcc-f' 'brcc-t' 'brlo-f' 'brlo-t' 'clr' 'cp' 'cpi' 'dec'
 'inc' 'ldi' 'mov' 'mul' 'rjmp' 'sbc' 'sub']


# Next we need to remove hyphins

In [17]:
for i in range(len(all_code_unique)):
    if "-t" in all_code_unique[i]:
        all_code_unique[i] = all_code_unique[i][:-2] + "T"
    elif "-f" in all_code_unique[i]:
        all_code_unique[i] = all_code_unique[i][:-2] + "F"

In [18]:
print(all_code_unique)

['adc' 'add' 'brccF' 'brccT' 'brloF' 'brloT' 'clr' 'cp' 'cpi' 'dec' 'inc'
 'ldi' 'mov' 'mul' 'rjmp' 'sbc' 'sub']


# Next, for tokenizer to work, we need it in a single string.

In [19]:
string_all_code_unique = ""
First = True
for inst in all_code_unique:
    if First:
        string_all_code_unique = string_all_code_unique + inst
        First = False
    else:
        string_all_code_unique = string_all_code_unique + ", " + inst
print(string_all_code_unique)

adc, add, brccF, brccT, brloF, brloT, clr, cp, cpi, dec, inc, ldi, mov, mul, rjmp, sbc, sub


In [20]:
temp = np.array([string_all_code_unique])

In [23]:
np.save(save_folder + "Code_for_Tokenizer.npy", temp)

# Next we tokenize the string to obtain the tokenizer

In [24]:
all_instructions = string_all_code_unique

sentence_Code = [all_instructions]
tokenizer.fit_on_texts(sentence_Code)

In [25]:
print(tokenizer.texts_to_sequences(sentence_Code))

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]]


In [27]:
print(tokenizer.texts_to_sequences(['brloF']))
print(tokenizer.texts_to_sequences(['ldi']))
print(tokenizer.texts_to_sequences(['adc']))
print(tokenizer.texts_to_sequences(['sbc']))

[[5]]
[[12]]
[[1]]
[[16]]


# Now we only need to load the Code_for_Tokenizer.npy, and tokenize.fit_on_text([Code_for_Tokenizer]) to obtain the exact same tokenizer.